In [1]:
# A Devito script aiming to replicate Fig 6 in Katou et al 2009

In [15]:
from devito import *
from examples.seismic.source import RickerSource, TimeAxis
from examples.seismic import plot_image, Model
import numpy as np

from sympy import init_printing, latex
init_printing(use_latex=True)

%matplotlib qt

In [16]:
extent = (400., 400.)
# Grid is 2km x 2km with spacing 10m
shape = (401, 401)
# Define x and z as spatial dimentions for Sympy
x = SpaceDimension(name='x', spacing=Constant(name='h_x', value=extent[0]/(shape[0]-1)))
# Dimension called x, with constant spacing of 10 (called h_x) 
z = SpaceDimension(name='z', spacing=Constant(name='h_z', value=extent[1]/(shape[1]-1)))
grid = Grid(extent=extent, shape=shape, dimensions=(x, z))
# Cartesian grid with dimensions of x and z. 201 x 201 gridpoints and measuring 2km x 2km

In [17]:
# Timestep size from Eq. 7 (Virieux 1986) with V_p=3500. and dx=5
t0, tn = 0., 180.
dt = (1. / np.sqrt(2.)) / 2500.
time_range = TimeAxis(start=t0, stop=tn, step=dt) #Set up time axis object for ricker source

src = RickerSource(name='src', grid=grid, f0=0.100, time_range=time_range) # Ricker wavelet source
src.coordinates.data[:] = [100., 100.] 
# 2 element array containing x and z positions of source
# Position is defined in meters, rather than grid points
src.show() # Show the source function

In [18]:
# Now we create the velocity and pressure fields
so = 2 # Spatial derivatives are second order accurate
# Grids are staggered to prevent pressure decoupling. Note that staggering is done in corresponding directions
vx= TimeFunction(name='vx', grid=grid, staggered=x, space_order=so) # Velocity field x
vz = TimeFunction(name='vz', grid=grid, staggered=z, space_order=so) # Velocity field z
# staggered=x entails discretization on x edges
txx = TimeFunction(name='txx', grid=grid, staggered=NODE, space_order=so)
tzz = TimeFunction(name='tzz', grid=grid, staggered=NODE, space_order=so)
txz = TimeFunction(name='txz', grid=grid, staggered=(x, z), space_order=so)
# Stress axis

In [19]:
# Now let's try and create the staggered updates
t = grid.stepping_dim
time = grid.time_dim

# We need some initial conditions
# Replace these with functions
V_p = Function(name="V_p", grid=grid, space_order=2)
V_p.data[:, 301:] = 2500. # Lower
V_p.data[:, :301] = 2000. # Upper
V_s = Function(name="V_s", grid=grid, space_order=2)
V_s.data[:, 301:] = 1500.
V_s.data[:, :301] = 1000.
rho = Function(name="rho", grid=grid, space_order=2)
rho.data[:, 301:] = 1/1900.
rho.data[:, :301] = 1/1500.

# print(rho.data[100,100]*V_p.data[100,100]**2-2*rho.data[100,100]*V_s.data[100,100]**2, rho.data[100, 100]*V_s.data[100,100]**2)

plot_image(V_p.data)

# Need a Model object to obtain damping terms
model = Model(vp=V_p, origin=(0., 0.), shape=shape, spacing=(1., 1.), space_order=2, nbpml=20)

# The source injection term
src_xx = src.inject(field=txx.forward, expr=src) 
src_zz = src.inject(field=tzz.forward, expr=src)


# fdelmodc reference implementation
u_vx = Eq(vx.forward, vx + dt*rho*(txx.dx + txz.dz)) #PDEs for updating all fields

u_vz = Eq(vz.forward, vz + dt*rho*(txz.dx + tzz.dz))

u_txx = Eq(txx.forward, txx + (rho*V_p**2)*dt * vx.forward.dx + (rho*V_p**2-2*rho*V_s**2)*dt * vz.forward.dz)
u_tzz = Eq(tzz.forward, tzz + (rho*V_p**2)*dt * vz.forward.dz + (rho*V_p**2-2*rho*V_s**2)*dt * vx.forward.dx)

u_txz = Eq(txz.forward, txz + (rho*V_s**2)*dt * (vx.forward.dz + vz.forward.dx))

In [20]:
op = Operator([u_vx, u_vz, u_txx, u_tzz, u_txz] + src_xx + src_zz) #Operator can iterate multiple PDEs at once
#Source is injected in xx and zz directions
#?op

In [21]:
# Reset the fields
vx.data[:] = 0. #Velocity components
vz.data[:] = 0.
txx.data[:] = 0. #Symmetric stress tensors
tzz.data[:] = 0.
txz.data[:] = 0.

op()

KeyboardInterrupt: 

In [11]:
# Let's see what we got....
#plot_image(vx.data[0], vmin=-.5*1e-2, vmax=.5*1e-2, cmap="seismic")
plot_image(vx.data[0], vmin=-.5*1e-2, vmax=.5*1e-2)


In [11]:
#plot_image(vz.data[0], vmin=-.5*1e-2, vmax=.5*1e-2, cmap="seismic")
plot_image(vz.data[0], vmin=-.5*1e-2, vmax=.5*1e-2)

In [74]:
#plot_image(txx.data[0], vmin=-.5*1e-2, vmax=.5*1e-2, cmap="seismic")
plot_image(txx.data[0], vmin=-.5*1e-2, vmax=.5*1e-2)

In [45]:
#plot_image(tzz.data[0], vmin=-.5*1e-2, vmax=.5*1e-2, cmap="seismic")
plot_image(tzz.data[0], vmin=-.5*1e-2, vmax=.5*1e-2)

In [24]:
#plot_image(txz.data[0], vmin=-.5*1e-2, vmax=.5*1e-2, cmap="seismic")
plot_image(txz.data[0], vmin=-.5*1e-2, vmax=.5*1e-2)